# Create Sentiment for Netmums

## Imports

In [1]:
import sqlite3
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pathlib import Path
from scraping import create_connection
from tqdm.notebook import tqdm
from math import floor

## File Locations

In [2]:
p = Path.cwd()
path_parent = p.parents[0]
path_db = str(path_parent / "database" / "netmums-merged.db")

## Functions

In [3]:
def get_size(conn):
    """ gets the size of the data set in number of rows
    :param conn: connection the the db
    :return size: size of the posts table
    """
    cur = conn.cursor()
    cur.execute(''' SELECT COUNT(id) FROM posts ''')
    size = cur.fetchone()
    if size:
        return int(size[0])
    raise SystemExit("No size found")

In [4]:
def process_data(chunksize):
    """ read data in chunks from the table, format the text,
        apply the sentiemnt analyzer, and write chunks to 
        the sentiment table
    :param chunksize: size of chunks
    """
    sql = ''' SELECT * FROM text '''
    reader = pd.read_sql_query(sql,
                               conn,
                               chunksize=chunksize)
    for i, df in enumerate(tqdm(reader)):
        df = gen_sentiment(df, 'text_clean', 'clean')
        df.drop('text_clean', axis=1, inplace=True)
        if i == 0:
            df.to_sql('sentiment', conn, if_exists='replace', index=False)
        else:
            df.to_sql('sentiment', conn, if_exists='append', index=False)

In [5]:
def gen_sentiment(df, var, name):
    """ apply the sentiment score to the input var
    :param var: string name of column getting sentiment for
    :param name: string variable suffix
    :return score: a dictionary of scores (neg, neu, pos, compound)
    """
    sentiment = df[var].apply(lambda x: sentiment_scores(x, analyzer))
    name_neg = "neg_sen_{}".format(name)
    name_neu = "neu_sen_{}".format(name)
    name_pos = "pos_sen_{}".format(name)
    name_com = "com_sen_{}".format(name)
    df[name_neg] = sentiment.apply(lambda x: x.get('neg', 0))
    df[name_neu] = sentiment.apply(lambda x: x.get('neu', 0))
    df[name_pos] = sentiment.apply(lambda x: x.get('pos', 0))
    df[name_com] = sentiment.apply(lambda x: x.get('compound', 0))
    del sentiment
    return df

In [6]:
def sentiment_scores(sentence, analyzer):
    """ create sentiment scores with the VADER analyzer
    :param sentence: sentence to create scores for
    :param analyzer: VADER sentiment analyzer
    :return score: a dictionary of scores (neg, neu, pos, compound)
    """
    score = analyzer.polarity_scores(sentence)
    return score

## Loop through chunks

In [7]:
conn = create_connection(path_db)
size = get_size(conn)
nchunks = 200
chunksize = floor(size / nchunks)

In [8]:
analyzer = SentimentIntensityAnalyzer()

In [9]:
process_data(chunksize)

|          | 0/? [00:00<?, ?it/s]

In [10]:
conn.commit()
conn.close()